This notebook is where model training occurs.

In [ ]:
!pip install datasets evaluate huggingface_hub transformers==4.28.0 rouge-score nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
!git config --global credential.helper store
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import numpy as np
import pandas as pd

from datasets import load_dataset

In [ ]:
dataset = load_dataset("mehnaazasad/arxiv_astro_co_ga")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/mehnaazasad___json/mehnaazasad--arxiv_astro_co_ga-11e8bca51d1e07b5/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'abstract'],
        num_rows: 79727
    })
    validation: Dataset({
        features: ['title', 'abstract'],
        num_rows: 9966
    })
    test: Dataset({
        features: ['title', 'abstract'],
        num_rows: 9966
    })
})

In [ ]:
abstract = dataset['train']['abstract'][0]

In [ ]:
abstract

'  We present a method to build a probability density function (pdf) for the age\nof a star based on its peculiar velocities $U$, $V$ and $W$ and its orbital\neccentricity. The sample used in this work comes from the Geneva-Copenhagen\nSurvey (GCS) which contains both the spatial velocities, orbital eccentricities\nand isochronal ages for about $14\\,000$ stars. Using the GCS stars, we fitted\nthe parameters that describe the relations between the distributions of\nkinematical properties and age. This parametrization allows us to obtain an age\nprobability from the kinematical data. From this age pdf, we estimate an\nindividual average age for the star using the most likely age and the expected\nage. We have obtained the stellar age pdf for the age of $9\\,102$ stars from\nthe GCS and have shown that the distribution of individual ages derived from\nour method is in good agreement with the distribution of isochronal ages. We\nalso observe a decline in the mean metallicity with our ages

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
model_checkpoint = 'facebook/bart-large'
tokenizer = BartTokenizer.from_pretrained(model_checkpoint)
model = BartForConditionalGeneration.from_pretrained(model_checkpoint)

In [ ]:
sample_inputs = tokenizer(abstract)

In [ ]:
tokenizer.convert_ids_to_tokens(sample_inputs.input_ids)[:20]

['<s>',
 'Ġ',
 'ĠWe',
 'Ġpresent',
 'Ġa',
 'Ġmethod',
 'Ġto',
 'Ġbuild',
 'Ġa',
 'Ġprobability',
 'Ġdensity',
 'Ġfunction',
 'Ġ(',
 'pdf',
 ')',
 'Ġfor',
 'Ġthe',
 'Ġage',
 'Ċ',
 'of']

In [ ]:
# Let's test with two abstracts

In [ ]:
abstracts = dataset['train']['abstract'][:2]

In [ ]:
abstracts

['  We present a method to build a probability density function (pdf) for the age\nof a star based on its peculiar velocities $U$, $V$ and $W$ and its orbital\neccentricity. The sample used in this work comes from the Geneva-Copenhagen\nSurvey (GCS) which contains both the spatial velocities, orbital eccentricities\nand isochronal ages for about $14\\,000$ stars. Using the GCS stars, we fitted\nthe parameters that describe the relations between the distributions of\nkinematical properties and age. This parametrization allows us to obtain an age\nprobability from the kinematical data. From this age pdf, we estimate an\nindividual average age for the star using the most likely age and the expected\nage. We have obtained the stellar age pdf for the age of $9\\,102$ stars from\nthe GCS and have shown that the distribution of individual ages derived from\nour method is in good agreement with the distribution of isochronal ages. We\nalso observe a decline in the mean metallicity with our age

In [ ]:
sample_inputs = tokenizer(abstracts, padding=True)

In [ ]:
tokenizer.convert_ids_to_tokens(sample_inputs.input_ids[0])

['<s>',
 'Ġ',
 'ĠWe',
 'Ġpresent',
 'Ġa',
 'Ġmethod',
 'Ġto',
 'Ġbuild',
 'Ġa',
 'Ġprobability',
 'Ġdensity',
 'Ġfunction',
 'Ġ(',
 'pdf',
 ')',
 'Ġfor',
 'Ġthe',
 'Ġage',
 'Ċ',
 'of',
 'Ġa',
 'Ġstar',
 'Ġbased',
 'Ġon',
 'Ġits',
 'Ġpeculiar',
 'Ġvel',
 'oc',
 'ities',
 'Ġ$',
 'U',
 '$,',
 'Ġ$',
 'V',
 '$',
 'Ġand',
 'Ġ$',
 'W',
 '$',
 'Ġand',
 'Ġits',
 'Ġorbital',
 'Ċ',
 'e',
 'cc',
 'entric',
 'ity',
 '.',
 'ĠThe',
 'Ġsample',
 'Ġused',
 'Ġin',
 'Ġthis',
 'Ġwork',
 'Ġcomes',
 'Ġfrom',
 'Ġthe',
 'ĠGeneva',
 '-',
 'C',
 'open',
 'h',
 'agen',
 'Ċ',
 'Sur',
 'vey',
 'Ġ(',
 'G',
 'CS',
 ')',
 'Ġwhich',
 'Ġcontains',
 'Ġboth',
 'Ġthe',
 'Ġspatial',
 'Ġvel',
 'oc',
 'ities',
 ',',
 'Ġorbital',
 'Ġeccentric',
 'ities',
 'Ċ',
 'and',
 'Ġis',
 'och',
 'ron',
 'al',
 'Ġages',
 'Ġfor',
 'Ġabout',
 'Ġ$',
 '14',
 '\\',
 ',',
 '000',
 '$',
 'Ġstars',
 '.',
 'ĠUsing',
 'Ġthe',
 'ĠG',
 'CS',
 'Ġstars',
 ',',
 'Ġwe',
 'Ġfitted',
 'Ċ',
 'the',
 'Ġparameters',
 'Ġthat',
 'Ġdescribe',
 'Ġthe',
 'Ġrelati

In [ ]:
titles = dataset['train']['title'][:2]
titles

['A method to estimate stellar ages from kinematical data',
 'Prediction of the Cosmic Evolution of the CO-Luminosity Functions']

In [ ]:
sample_outputs = tokenizer(titles, padding=True)
sample_outputs
tokenizer.convert_ids_to_tokens(sample_outputs.input_ids[0])

['<s>',
 'A',
 'Ġmethod',
 'Ġto',
 'Ġestimate',
 'Ġstellar',
 'Ġages',
 'Ġfrom',
 'Ġk',
 'inem',
 'atical',
 'Ġdata',
 '</s>',
 '<pad>',
 '<pad>',
 '<pad>']

In [ ]:
#Switching to using FastTokenizer since it is faster when used alongside batched=True
from transformers import BartTokenizerFast

In [ ]:
tokenizer = BartTokenizerFast.from_pretrained(model_checkpoint)

In [ ]:
tokenizer.convert_ids_to_tokens(sample_inputs.input_ids[0])

['<s>',
 'Ġ',
 'ĠWe',
 'Ġpresent',
 'Ġa',
 'Ġmethod',
 'Ġto',
 'Ġbuild',
 'Ġa',
 'Ġprobability',
 'Ġdensity',
 'Ġfunction',
 'Ġ(',
 'pdf',
 ')',
 'Ġfor',
 'Ġthe',
 'Ġage',
 'Ċ',
 'of',
 'Ġa',
 'Ġstar',
 'Ġbased',
 'Ġon',
 'Ġits',
 'Ġpeculiar',
 'Ġvel',
 'oc',
 'ities',
 'Ġ$',
 'U',
 '$,',
 'Ġ$',
 'V',
 '$',
 'Ġand',
 'Ġ$',
 'W',
 '$',
 'Ġand',
 'Ġits',
 'Ġorbital',
 'Ċ',
 'e',
 'cc',
 'entric',
 'ity',
 '.',
 'ĠThe',
 'Ġsample',
 'Ġused',
 'Ġin',
 'Ġthis',
 'Ġwork',
 'Ġcomes',
 'Ġfrom',
 'Ġthe',
 'ĠGeneva',
 '-',
 'C',
 'open',
 'h',
 'agen',
 'Ċ',
 'Sur',
 'vey',
 'Ġ(',
 'G',
 'CS',
 ')',
 'Ġwhich',
 'Ġcontains',
 'Ġboth',
 'Ġthe',
 'Ġspatial',
 'Ġvel',
 'oc',
 'ities',
 ',',
 'Ġorbital',
 'Ġeccentric',
 'ities',
 'Ċ',
 'and',
 'Ġis',
 'och',
 'ron',
 'al',
 'Ġages',
 'Ġfor',
 'Ġabout',
 'Ġ$',
 '14',
 '\\',
 ',',
 '000',
 '$',
 'Ġstars',
 '.',
 'ĠUsing',
 'Ġthe',
 'ĠG',
 'CS',
 'Ġstars',
 ',',
 'Ġwe',
 'Ġfitted',
 'Ċ',
 'the',
 'Ġparameters',
 'Ġthat',
 'Ġdescribe',
 'Ġthe',
 'Ġrelati

In [ ]:
# Set up a function that takes in a sample (abstract+title) and handles tokenization and truncation
max_input_length = 1024
max_target_length = 128

def preprocess_function(samples):
    inputs = samples['abstract']
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    outputs = samples['title']
    labels = tokenizer(text_target=outputs, max_length=max_target_length, truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [ ]:

preprocess_function(dataset['train'][0])

{'input_ids': [0, 1437, 166, 1455, 10, 5448, 7, 1119, 10, 18102, 16522, 5043, 36, 31494, 43, 13, 5, 1046, 50118, 1116, 10, 999, 716, 15, 63, 28178, 23021, 1975, 2192, 68, 791, 47110, 68, 846, 1629, 8, 68, 771, 1629, 8, 63, 38767, 50118, 242, 7309, 44581, 1571, 4, 20, 7728, 341, 11, 42, 173, 606, 31, 5, 11559, 12, 347, 12592, 298, 10955, 50118, 27526, 9932, 36, 534, 6842, 43, 61, 6308, 258, 5, 34999, 23021, 1975, 2192, 6, 38767, 29555, 2192, 50118, 463, 16, 4306, 2839, 337, 4864, 13, 59, 68, 1570, 37457, 6, 151, 1629, 2690, 4, 8630, 5, 272, 6842, 2690, 6, 52, 15898, 50118, 627, 17294, 14, 6190, 5, 3115, 227, 5, 26070, 9, 50118, 4245, 991, 36105, 3611, 8, 1046, 4, 152, 40206, 594, 21645, 1258, 2386, 201, 7, 6925, 41, 1046, 50118, 4892, 428, 4484, 31, 5, 449, 10748, 36105, 414, 4, 1740, 42, 1046, 46559, 6, 52, 3278, 41, 50118, 42333, 674, 1046, 13, 5, 999, 634, 5, 144, 533, 1046, 8, 5, 421, 50118, 1580, 4, 166, 33, 4756, 5, 13641, 1046, 46559, 13, 5, 1046, 9, 68, 466, 37457, 6, 17024, 162

In [ ]:
tokenizer.convert_ids_to_tokens(preprocess_function(dataset['train'][0]).input_ids)
tokenizer.convert_ids_to_tokens(preprocess_function(dataset['train'][0]).labels)

['<s>',
 'A',
 'Ġmethod',
 'Ġto',
 'Ġestimate',
 'Ġstellar',
 'Ġages',
 'Ġfrom',
 'Ġk',
 'inem',
 'atical',
 'Ġdata',
 '</s>']

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/79727 [00:00<?, ? examples/s]

Map:   0%|          | 0/9966 [00:00<?, ? examples/s]

Map:   0%|          | 0/9966 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
import nltk
from datasets import load_metric

In [ ]:
metric_old = load_metric("rouge")

<ipython-input-30-3695b07a4ab8>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_old = load_metric("rouge")


In [ ]:
metric_old

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [ ]:
from evaluate import load

In [ ]:
metric_new = load("rouge")
metric_new

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [ ]:
titles_pred = ['A method to estimate stellar ages from kinematical data','Prediction of the Cosmic Evolution of the CO-Luminosity Functions']
titles_ref = titles
titles

['A method to estimate stellar ages from kinematical data',
 'Prediction of the Cosmic Evolution of the CO-Luminosity Functions']

In [ ]:
result = metric_old.compute(predictions=titles_pred, references=titles_ref, use_stemmer=True, use_aggregator=True)
result

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

In [ ]:
result = metric_new.compute(predictions=titles_pred, references=titles_ref, use_stemmer=True, use_aggregator=True)
result

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

Creating a baseline model using the first 1-3 sentences of the abstract and the test dataset for evaluation

In [ ]:
dataset['train']['abstract'][0]

'  We present a method to build a probability density function (pdf) for the age\nof a star based on its peculiar velocities $U$, $V$ and $W$ and its orbital\neccentricity. The sample used in this work comes from the Geneva-Copenhagen\nSurvey (GCS) which contains both the spatial velocities, orbital eccentricities\nand isochronal ages for about $14\\,000$ stars. Using the GCS stars, we fitted\nthe parameters that describe the relations between the distributions of\nkinematical properties and age. This parametrization allows us to obtain an age\nprobability from the kinematical data. From this age pdf, we estimate an\nindividual average age for the star using the most likely age and the expected\nage. We have obtained the stellar age pdf for the age of $9\\,102$ stars from\nthe GCS and have shown that the distribution of individual ages derived from\nour method is in good agreement with the distribution of isochronal ages. We\nalso observe a decline in the mean metallicity with our ages

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
"\n".join(sent_tokenize(dataset['test']['abstract'][0])[:3])

"  We introduce a very early universe model based on the thermodynamics of a gas\nof closed strings in a background which is non-perturbative in $\\alpha'$.\nUpon\nconsidering the fully $\\alpha'$-corrected equations extended to include certain\nanisotropic cosmological backgrounds, we describe the evolution of the system\nin three different stages parametrized by the gas' equation of state.\nUsing\nstandard string thermodynamical arguments, we start with an isotropic\n10-dimensional universe inside the string scale and evolve it towards a\nuniverse with four large spacetime dimensions and six stabilized internal\ndimensions in the Einstein frame."

In [ ]:
ref_summaries = dataset['test']['title'][:20]
for i in range(3):
    pred_summaries = list(map(lambda x: "\n".join(sent_tokenize(x)[:i+1]), dataset['test']['abstract'][:20]))
    scores = metric_new.compute(predictions=pred_summaries, references=ref_summaries, use_stemmer=True, use_aggregator=True)
    # result_median = {key: value.mid.fmeasure * 100 for key, value in scores.items()}
    result = {key: value * 100 for key, value in scores.items()}
    result = {k: round(v,4) for k, v in result.items()}
    print("First {0} sentence(s): Scores {1}".format(i+1, result))

First 1 sentence(s): Scores {'rouge1': 33.5817, 'rouge2': 15.5748, 'rougeL': 24.6671, 'rougeLsum': 30.1371}
First 2 sentence(s): Scores {'rouge1': 25.0404, 'rouge2': 11.201, 'rougeL': 19.0928, 'rougeLsum': 23.6574}
First 3 sentence(s): Scores {'rouge1': 20.6268, 'rouge2': 10.7274, 'rougeL': 16.0666, 'rougeLsum': 19.9266}


In [ ]:
result

{'rouge1': 20.6268, 'rouge2': 10.7274, 'rougeL': 16.0666, 'rougeLsum': 19.9266}

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated titles into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them (-100 come from data collator padding tokens)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference titles into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Compute ROUGE scores
    result = metric_new.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v,4) for k, v in result.items()}


In [ ]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
training_args = Seq2SeqTrainingArguments(
    output_dir = f"{model_name}-finetuned-arxiv-co-ga-latest",
    # overwrite_output_dir = True, # Needed because restarted training. Set to False if using checkpoints.
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    hub_strategy = "checkpoint",
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/mehnaazasad/bart-large-finetuned-arxiv-co-ga-latest into local empty directory.


Download file last-checkpoint/optimizer.pt:   0%|          | 16.4k/3.03G [00:00<?, ?B/s]

Download file runs/Jun06_21-56-32_e1a7faa0416b/events.out.tfevents.1686088641.e1a7faa0416b.3052.0: 100%|######…

Download file runs/Jun06_21-56-32_e1a7faa0416b/1686088641.0410676/events.out.tfevents.1686088641.e1a7faa0416b.…

Download file runs/Jun07_23-59-22_2cf7817a9fbd/events.out.tfevents.1686183945.2cf7817a9fbd.350.0: 100%|#######…

...

Clean file last-checkpoint/scaler.pt: 100%|##########| 557/557 [00:00<?, ?B/s]

Download file last-checkpoint/pytorch_model.bin:   0%|          | 1.39k/1.51G [00:00<?, ?B/s]

Clean file last-checkpoint/pytorch_model.bin:   0%|          | 1.00k/1.51G [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/1.51G [00:00<?, ?B/s]

Clean file last-checkpoint/optimizer.pt:   0%|          | 1.00k/3.03G [00:00<?, ?B/s]

In [ ]:
f"{model_name}-finetuned-arxiv-co-ga-latest/last-checkpoint"

'bart-large-finetuned-arxiv-co-ga-latest/last-checkpoint'

In [ ]:
trainer.train(resume_from_checkpoint=f"{model_name}-finetuned-arxiv-co-ga-latest/last-checkpoint")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,1.775200,1.718963,43.891600,23.629600,38.229000,39.351900


TrainOutput(global_step=9966, training_loss=0.26252660535077077, metrics={'train_runtime': 1366.0476, 'train_samples_per_second': 58.363, 'train_steps_per_second': 7.295, 'total_flos': 4.461808915764019e+16, 'train_loss': 0.26252660535077077, 'epoch': 1.0})

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/1.51G [00:00<?, ?B/s]

Upload file runs/Jun09_00-26-13_117ce26b8051/events.out.tfevents.1686271844.117ce26b8051.1152.0:   0%|        …

To https://huggingface.co/mehnaazasad/bart-large-finetuned-arxiv-co-ga-latest
   8107b1f..be6b753  main -> main

   8107b1f..be6b753  main -> main

To https://huggingface.co/mehnaazasad/bart-large-finetuned-arxiv-co-ga-latest
   be6b753..c1f6dc3  main -> main

   be6b753..c1f6dc3  main -> main



'https://huggingface.co/mehnaazasad/bart-large-finetuned-arxiv-co-ga-latest/commit/be6b75380e8e18a0ccbbde549f0ef2cfac813afe'

In [ ]:
!nvidia-smi

Thu Jun  8 20:48:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    48W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !pip install GPUtil

In [ ]:

# Code used for emptying GPU cache
# import torch
# from GPUtil import showUtilization as gpu_usage
# from numba import cuda

# def free_gpu_cache():
#     print("Initial GPU Usage")
#     gpu_usage()

#     torch.cuda.empty_cache()

#     cuda.select_device(0)
#     cuda.close()
#     cuda.select_device(0)

#     print("GPU Usage after emptying the cache")
#     gpu_usage()

# free_gpu_cache()